In [12]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import DBSCAN
# from sklearn.neighbors import NearestNeighbors
# import os
# from datetime import datetime

# # مسیر فایل ورودی و خروجی
# INPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"
# OUTPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\dbscan_lube_oil_system_anomalies_g11.xlsx"

# # ستون‌های عددی برای مدل DBSCAN
# FEATURE_COLUMNS = [
#     'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
#     'AssetID_8346', 'AssetID_9286', 'AssetID_9287'
# ]

# # ستون‌های نهایی خروجی
# OUTPUT_COLUMNS = [
#     'id', 'model_name', 'model_result', 'model_value', 'cluster_label',
#     'DateTime', 'DateOnly', 'RecordDate',
#     'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
#     'AssetID_8346', 'AssetID_9286', 'AssetID_9287',
#     'RecordTime'
# ]

# # خواندن داده از اکسل
# print("در حال خواندن فایل ورودی...")
# df = pd.read_excel(INPUT_FILE)

# # اطمینان از وجود ستون‌های لازم
# required_cols = ['id', 'RecordDate', 'RecordTime'] + FEATURE_COLUMNS
# missing = [col for col in required_cols if col not in df.columns]
# if missing:
#     raise ValueError(f"ستون‌های زیر در فایل وجود ندارند: {missing}")

# # آماده‌سازی داده‌ها
# df_clean = df[required_cols].dropna().copy()
# df_clean = df_clean.reset_index(drop=True)

# # استانداردسازی داده‌ها
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df_clean[FEATURE_COLUMNS])

# # اجرای DBSCAN
# print("در حال اجرای DBSCAN...")
# dbscan = DBSCAN(eps=0.5, min_samples=5)  # تنظیمات مناسب برای داده‌های صنعتی
# cluster_labels = dbscan.fit_predict(X_scaled)

# df_clean['cluster_label'] = cluster_labels

# # جدا کردن انومالی‌ها (برچسب -1)
# anomalies = df_clean[df_clean['cluster_label'] == -1].copy()

# if len(anomalies) == 0:
#     print("هیچ انومالی تشخیص داده نشد!")
# else:
#     print(f"{len(anomalies)} انومالی تشخیص داده شد.")

# # محاسبه مرکز خوشه‌های نرمال (غیر -1)
# normal_centers = []
# normal_labels = [lbl for lbl in np.unique(cluster_labels) if lbl != -1]

# for label in normal_labels:
#     cluster_points = X_scaled[cluster_labels == label]
#     if len(cluster_points) > 0:
#         center = cluster_points.mean(axis=0)
#         normal_centers.append(center)

# # اگر خوشه نرمال وجود داشت → محاسبه فاصله تا نزدیک‌ترین مرکز
# if len(normal_centers) > 0:
#     normal_centers = np.array(normal_centers)
#     nbrs = NearestNeighbors(n_neighbors=1).fit(normal_centers)
#     distances, _ = nbrs.kneighbors(X_scaled[cluster_labels == -1])
#     dist_to_center = distances.flatten()

#     # تبدیل فاصله به امتیاز 0 تا 100 (هرچه دورتر → امتیاز بالاتر)
#     max_dist = np.percentile(dist_to_center, 95) if len(dist_to_center) > 0 else 1
#     anomaly_scores = np.clip(100 * (dist_to_center / (max_dist + 1e-8)), 0, 100)
# else:
#     # اگر هیچ خوشه نرمالی نبود، امتیاز بالا بده
#     anomaly_scores = np.array([100.0] * len(anomalies))

# anomalies['model_value'] = np.round(anomaly_scores, 2)

# # اضافه کردن ستون‌های ثابت
# anomalies['model_name'] = 'DBSCAN_Anomaly_SmartScore'
# anomalies['model_result'] = 'Anomaly'

# # ساخت ستون DateTime از RecordDate + RecordTime
# anomalies['DateTime'] = pd.to_datetime(anomalies['RecordDate'].astype(str) + ' ' + anomalies['RecordTime'].astype(str))
# anomalies['DateOnly'] = anomalies['DateTime'].dt.date

# # تبدیل DateTime به عدد روز اکسل (مثل نمونه‌ای که دادی)
# anomalies['DateTime'] = anomalies['DateTime'].apply(lambda x: x.timestamp() / 86400 + 25569)  # تبدیل به فرمت اکسل

# # مرتب‌سازی بر اساس شدت انومالی (نزولی)
# anomalies = anomalies.sort_values(by='model_value', ascending=False).reset_index(drop=True)

# # انتخاب و مرتب کردن ستون‌های نهایی
# final_df = anomalies[OUTPUT_COLUMNS].copy()

# # ذخیره در اکسل
# final_df.to_excel(OUTPUT_FILE, index=False)

# print("\nکار تمام شد!")
# print(f"تعداد انومالی‌های تشخیص داده شده: {len(final_df)}")
# print(f"فایل خروجی ذخیره شد در:")
# print(f"→ {OUTPUT_FILE}")
# print("\nنمونه ۵ انومالی اول:")
# print(final_df.head()[['id', 'RecordDate', 'RecordTime', 'model_value', 'AssetID_8341', 'AssetID_8343']].to_string(index=False))

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.neighbors import NearestNeighbors
import os

# مسیر صحیح فایل ورودی (غلط املایی برطرف شد!)
INPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"

# مسیر خروجی‌ها — دقیقاً با نام ثابت و بدون تاریخ
OUTPUT_FOLDER = r"C:\BI\KZCCPP\G11\Lube Oil System"

DBSCAN_OUTPUT = os.path.join(OUTPUT_FOLDER, "dbscan_lube_oil_system_anomalies_g11.xlsx")
FINAL_OUTPUT  = os.path.join(OUTPUT_FOLDER, "dbscan_one_class_svm_lube_oil_system_anomalies_g11.xlsx")

# فایل‌های موقت برای جلوگیری از خطای Permission
TEMP_DBSCAN = DBSCAN_OUTPUT.replace(".xlsx", "_temp.xlsx")
TEMP_FINAL  = FINAL_OUTPUT.replace(".xlsx", "_temp.xlsx")

FEATURE_COLUMNS = ['AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
                   'AssetID_8346', 'AssetID_9286', 'AssetID_9287']

OUTPUT_COLUMNS = ['id', 'model_name', 'model_result', 'model_value', 'cluster_label',
                  'DateTime', 'DateOnly', 'RecordDate',
                  'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
                  'AssetID_8346', 'AssetID_9286', 'AssetID_9287', 'RecordTime']

print("شروع تشخیص انومالی دو مرحله‌ای: DBSCAN + One-Class SVM\n")

# مرحله ۱: خواندن فایل
print("۱. در حال خواندن فایل ورودی...")
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"فایل پیدا نشد!\nمسیر بررسی شود:\n{INPUT_FILE}")

df = pd.read_excel(INPUT_FILE)
print(f"   فایل با موفقیت خوانده شد → {len(df)} رکورد")

required = ['id', 'RecordDate', 'RecordTime'] + FEATURE_COLUMNS
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"ستون‌های زیر در فایل وجود ندارند: {missing}")

df_clean = df[required].dropna().reset_index(drop=True)
X_scaled = StandardScaler().fit_transform(df_clean[FEATURE_COLUMNS])

# DBSCAN
print("   اجرای DBSCAN...")
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels = dbscan.fit_predict(X_scaled)
df_clean['cluster_label'] = labels

anomalies_dbscan = df_clean[df_clean['cluster_label'] == -1].copy()
if len(anomalies_dbscan) == 0:
    print("هیچ انومالی توسط DBSCAN تشخیص داده نشد.")
    exit()

print(f"   DBSCAN → {len(anomalies_dbscan)} انومالی پیدا کرد.")

# محاسبه model_value بر اساس فاصله از نزدیک‌ترین خوشه نرمال
normal_centers = [X_scaled[labels == l].mean(axis=0) for l in set(labels) if l != -1]
if normal_centers:
    normal_centers = np.array(normal_centers)
    nbrs = NearestNeighbors(n_neighbors=1).fit(normal_centers)
    distances, _ = nbrs.kneighbors(X_scaled[labels == -1])
    max_d = np.percentile(distances, 95) if len(distances) > 3 else distances.max()
    scores = np.clip(100 * distances.flatten() / (max_d + 1e-8), 0, 100)
else:
    scores = np.array([100.0] * len(anomalies_dbscan))

anomalies_dbscan['model_value'] = np.round(scores, 2)
anomalies_dbscan['model_name'] = 'DBSCAN_Anomaly_SmartScore'
anomalies_dbscan['model_result'] = 'Anomaly'

# ساخت DateTime به فرمت عدد اکسل
anomalies_dbscan['DateTime_dt'] = pd.to_datetime(
    anomalies_dbscan['RecordDate'].astype(str) + ' ' + anomalies_dbscan['RecordTime'].astype(str),
    errors='coerce'
)
anomalies_dbscan['DateTime'] = anomalies_dbscan['DateTime_dt'].apply(
    lambda x: x.timestamp() / 86400 + 25569 if pd.notnull(x) else np.nan
)
anomalies_dbscan['DateOnly'] = anomalies_dbscan['DateTime_dt'].dt.date

# ذخیره فایل DBSCAN
dbscan_result = anomalies_dbscan[OUTPUT_COLUMNS].sort_values('model_value', ascending=False)
dbscan_result.to_excel(TEMP_DBSCAN, index=False)
os.replace(TEMP_DBSCAN, DBSCAN_OUTPUT) if os.path.exists(DBSCAN_OUTPUT) else os.rename(TEMP_DBSCAN, DBSCAN_OUTPUT)
print(f"   خروجی DBSCAN ذخیره شد: dbscan_lube_oil_system_anomalies_g11.xlsx")

# مرحله ۲: One-Class SVM
print("\n۲. تأیید نهایی با One-Class SVM...")
X_normal = X_scaled[labels != -1]

if len(X_normal) < 10:
    print("   داده نرمال کافی نیست → خروجی نهایی = خروجی DBSCAN")
    final_result = dbscan_result.copy()
else:
    oc_svm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
    oc_svm.fit(X_normal)
    X_anom = X_scaled[labels == -1]
    svm_pred = oc_svm.predict(X_anom)
    
    confirmed = anomalies_dbscan[svm_pred == -1].copy()
    
    if len(confirmed) == 0:
        print("   One-Class SVM هیچ موردی را تأیید نکرد → خروجی نهایی = خروجی DBSCAN")
        final_result = dbscan_result.copy()
    else:
        svm_scores = oc_svm.decision_function(X_anom)
        svm_norm = 100 * (1 - (svm_scores[svm_pred == -1] - svm_scores.min()) / (svm_scores.ptp() + 1e-8))
        combined = (confirmed['model_value'].values + svm_norm) / 2
        confirmed['model_value'] = np.round(combined, 2)
        confirmed['model_name'] = 'DBSCAN_OneClassSVM_SmartScore'
        final_result = confirmed[OUTPUT_COLUMNS].sort_values('model_value', ascending=False)
        print(f"   {len(final_result)} انومالی توسط هر دو مدل تأیید شد.")

# ذخیره نهایی
final_result.to_excel(TEMP_FINAL, index=False)
if os.path.exists(FINAL_OUTPUT):
    os.replace(TEMP_FINAL, FINAL_OUTPUT)
else:
    os.rename(TEMP_FINAL, FINAL_OUTPUT)

# پایان
print("\n" + "="*85)
print("                  تمام! دو فایل با نام ثابت تولید شد")
print("="*85)
print(f"   ۱. dbscan_lube_oil_system_anomalies_g11.xlsx        → {len(dbscan_result)} مورد")
print(f"   ۲. dbscan_one_class_svm_lube_oil_system_anomalies_g11.xlsx → {len(final_result)} مورد")
print("="*85)

شروع تشخیص انومالی دو مرحله‌ای: DBSCAN + One-Class SVM

۱. در حال خواندن فایل ورودی...
   فایل با موفقیت خوانده شد → 10927 رکورد
   اجرای DBSCAN...
   DBSCAN → 295 انومالی پیدا کرد.
   خروجی DBSCAN ذخیره شد: dbscan_lube_oil_system_anomalies_g11.xlsx

۲. تأیید نهایی با One-Class SVM...
   205 انومالی توسط هر دو مدل تأیید شد.

                  تمام! دو فایل با نام ثابت تولید شد
   ۱. dbscan_lube_oil_system_anomalies_g11.xlsx        → 295 مورد
   ۲. dbscan_one_class_svm_lube_oil_system_anomalies_g11.xlsx → 205 مورد
